# Problem 1.(10 points)

In class, we implemented solving the Traveling Salesman Problem by training an RNN+MLP model by cross-entropy method. In this problem, we follow the idea from [A Tutorial on the Cross-Entropy Method](https://web.mit.edu/6.454/www/www_fall_2003/gew/CEtutorial.pdf) (pp. 25-27) by de Boer, Kroese, Mannor, and Rubinstein to simply use a Markov chain to generate tours. That is, each next city in a tour will be generated randomly based solely on the current city (as opposed to the entirety of the currently built part of the tour), via a matrix of probabilities.

Instead of directly implementing the procedure in the paper, to minimize work, we'll use our current model but replace the RNN+FFNN model with a single linear layer with one-hot input vectors (corresponding to the current city) and output giving scores of the potential next cities. This will correspond (after we apply softmax in the method that builds a tour based on scores) to multiplying by probability matrix. The actual difference with the paper is that they give an explicit formula for updating probabilities, whereas we will just use the optimizer that we already have.

Run the cells below to initialize the process.


In [ ]:
# instal library for computing exact solution of tsp (a bit faster than bruteforce) and a quick heuristic solution
!pip install python-tsp

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import random
import math

from python_tsp.exact import solve_tsp_dynamic_programming
from python_tsp.heuristics import solve_tsp_simulated_annealing

import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Large weight
INF_WEIGHT = 10**5

Graph generator

In [ ]:
def generate_graph_with_min_degree_2(num_vertices, prob=0.5):
  """Generates a graph where each vertex has a degree of at least 2."""
  graph = nx.Graph()
  for i in range(num_vertices):
      graph.add_node(i)

  # Random edges with probability prob
  for i in range(num_vertices):
    for j in range(i + 1, num_vertices):
      if random.random() < prob:  # Add an edge with probability prob
        weight = random.randint(1, 100)
        graph.add_edge(i, j, weight=weight)

  # Ensure min degree 2
  for i in range(num_vertices):
    if graph.degree[i] < 2:
      neighbors = list(graph.neighbors(i))
      if len(neighbors) == 0:
        other_nodes = [n for n in graph.nodes if n != i]
        if len(other_nodes) >= 2:
          weight = random.randint(1, 100)
          graph.add_edge(i, other_nodes[0], weight=weight)
          weight = random.randint(1, 100)
          graph.add_edge(i, other_nodes[1], weight=weight)
      elif len(neighbors) == 1:
        other_nodes = [n for n in graph.nodes if n != i and n not in neighbors]
        if len(other_nodes) >= 1:
          weight = random.randint(1, 100)
          graph.add_edge(i, other_nodes[0], weight=weight)

  return graph




def draw_graph(graph, inf_weight=INF_WEIGHT):
  """Draws the graph."""
  plt.figure(figsize=(8, 4))
  pos = nx.spring_layout(graph)

  # No edges with INF_WEIGHT
  edges_to_draw = [(u, v) for u, v, data in graph.edges(data=True) if data['weight'] != inf_weight]
  labels = {(u, v): data['weight'] for u, v, data in graph.edges(data=True) if data['weight'] != inf_weight}

  nx.draw(graph, pos, with_labels=True, node_color='skyblue', node_size=500, edgelist=edges_to_draw,
          edge_cmap=plt.cm.Blues, width=2)
  nx.draw_networkx_edge_labels(graph, pos, edge_labels=labels)
  plt.show()


def get_weight_matrix(graph):
  """Returns the weight matrix of the graph."""
  num_vertices = len(graph.nodes)
  weight_matrix = np.full((num_vertices, num_vertices), 10**5)  # Initialize with large weights

  for u, v, data in graph.edges(data=True):
    weight_matrix[u][v] = weight_matrix[v][u] = data['weight']

  for i in range(num_vertices):
    weight_matrix[i][i] = 0

  return weight_matrix

Generated graph

In [ ]:
# Fix the random seed for reproducibility
random.seed(42)
np.random.seed(42)

# Generate a random graph with 7 vertices
num_vertices = 9
graph = generate_graph_with_min_degree_2(num_vertices, prob = 0.7)

# Draw the graph
draw_graph(graph)

# Print the weight matrix
weight_matrix = get_weight_matrix(graph)
print("Weight Matrix:")
weight_matrix

A simple function that computes the total length of a tour.

In [ ]:
def compute_tour_length(tour, dist):
    # Compute the total length of a tour
    total_length = 0
    for i in range(len(tour)):
        start = tour[i]
        end = tour[(i + 1) % len(tour)]
        total_length += dist[start][end]
    return total_length

##Problem 1a.(2 points)
 Alter the code from the lecture so that ``TSPSolver`` only has a single linear layer (input features = ``num_cities``, output features = ``num_cities``). You'll need to adjust ``__init__``, ``forward`` methods, and the input tensor in the main loop of the ``sample_tour`` method.

In [ ]:
class TSPSolver(nn.Module):
    def __init__(self, num_cities, input_dim):
        super(TSPSolver, self).__init__()
        ### your code instead of commented out lines below:
        # self.encoder = RNNEncoder(input_dim, hidden_dim)
        # self.fc1 = nn.Linear(hidden_dim, hidden_dim)
        # self.fc2 = nn.Linear(hidden_dim, num_cities)
        self.num_cities = num_cities

    def forward(self, input, hx=None):
        input = input.view(input.size(0), -1)
        logits = ### your code here
        return logits

    def sample_tour(self, weight_matrix, start_vertex=0, temperature=1.0):
        # Sampling a complete tour using the model's policy

        tour = [start_vertex]
        remaining = list(range(self.num_cities))
        remaining.remove(start_vertex)


        city = start_vertex

        for _ in range(self.num_cities - 1):
            # Sample from current probabilities
            if len(remaining) == 1:
                city = remaining[0]
            else:
                ### your code
                one_hot_tensor =
                logits = self(one_hot_tensor)
                # Get the correct shape of probs
                probs = F.softmax(logits / temperature, dim=-1).squeeze(0) # squeezing to remove the batch dimension
                city_probs = torch.zeros(len(remaining))

                for i, r in enumerate(remaining):
                    # Index probs correctly
                    city_probs[i] = probs[r]
                city_probs /= city_probs.sum()
                city = remaining[torch.multinomial(city_probs, 1).item()]


            tour.append(city)
            remaining.remove(city)

        return tour

##Problem 1b.(2 points)
Adjust the code below to account for the new model. In particular, you'll need to adjust ``input_sequence``.

In [ ]:
def cross_entropy_method(solver, weight_matrix, optimizer,
                         num_iterations=10, population_size=100, elite_fraction=0.1):

    if population_size * elite_fraction < 1:
        raise ValueError("Population size times elite fraction should be at least 1.")

    # shape (num_cities, num_cities)
    weight_matrix_tensor = torch.tensor(weight_matrix, dtype=torch.float32)

    best_tour = None
    best_length = float('inf')

    # Define loss function
    criterion = nn.CrossEntropyLoss()

    for iteration in range(num_iterations):
        # Generate a population of tours
        tours = []
        tour_lengths = []

        for _ in range(population_size):
            tour = solver.sample_tour(weight_matrix_tensor) # list with len = num_cities
            tour_length = compute_tour_length(tour, weight_matrix)

            tours.append(tour)
            tour_lengths.append(tour_length)


        # Select elite tours
        num_elite = int(population_size * elite_fraction)
        elite_indices = np.argsort(tour_lengths)[:num_elite]
        elite_tours = [tours[i] for i in elite_indices]
        worst_elite_tour=tour_lengths[elite_indices[-1]]
        best_elite_tour=tour_lengths[elite_indices[0]]

        # Update best solution
        current_best_length = min(tour_lengths)
        if current_best_length < best_length:
            best_length = current_best_length
            best_tour = tours[np.argmin(tour_lengths)]

        # Train the model on elite tours
        if elite_tours:
            # Reset gradients
            solver.zero_grad()


            for tour in elite_tours:
                for j in range(len(tour)-1):
                    # Prepare input data
                    # Creating one-hot encoded vector of current city in tour
                    # your code below
                    input_sequence =

                    # target_city is the next city in tour
                    target_city = torch.tensor(tour[(j + 1) % len(tour)], dtype=torch.long)

                    # Forward pass
                    output = solver(input_sequence)

                    # Calculate loss
                    loss = criterion(output.squeeze(0), target_city)

                    # Backward pass and optimization
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

        if (iteration + 1) % 1 == 0:
            print(f"Iteration [{iteration+1}/{num_iterations}]")
            print(f"Worst Elite Tour: {worst_elite_tour}, Best Elite Tour: {best_elite_tour}")
            print(f"Current Best Tour: {best_tour}")
            print(f"Current Tour Length: {best_length}")
        if worst_elite_tour == best_elite_tour == best_length:
            print("Unlikely to learn anything new, stopping early.")
            break

    return best_tour, best_length

Our model succefully solves the TSP for this small graph.

### `CE_TSP` Function Overview

The `CE_TSP` function applies the **Cross-Entropy Method (CEM)** to solve instances of the **Traveling Salesman Problem (TSP)** on randomly generated graphs.



#### Solver Modes

##### `tsp_solver=True`
Uses a **dynamic programming** approach to compute the exact optimal solution to the TSP.  
This method is based on the classic **Held-Karp algorithm** [1], which has time complexity $\mathcal{O}(n^2 \cdot 2^n)$.  
⚠️ **Note:** This solver is only feasible for small graphs (typically fewer than 12 cities).

##### `tsp_heuristics=True`
Uses **simulated annealing** [2], a probabilistic metaheuristic inspired by physical annealing processes, to quickly find a **near-optimal tour**.  
This approach is significantly faster than exact solvers and serves as a practical baseline for larger instances.

---

#### Output

After training, the function prints:
- The best tour found by the CEM solver and its total cost.
- Optionally: the best tour and cost found by the exact and/or heuristic solvers (for comparison).

Use this setup to explore how well the Cross-Entropy Method can approximate or even outperform traditional methods on random TSP instances.

---

#### References

[1] Held, M., & Karp, R. M. (1962). A Dynamic Programming Approach to Sequencing Problems. *Journal of the Society for Industrial and Applied Mathematics*, 10(1), 196–210.  
[2] Kirkpatrick, S., Gelatt, C. D., & Vecchi, M. P. (1983). Optimization by Simulated Annealing. *Science*, 220(4598), 671–680.

In [ ]:
def CE_TSP(num_cities=8, start_city=0, prob = 0.8, iters=20, pop=1000, fraction = 0.1,
           lr=0.01, tsp_solver = False, tsp_heuristics = False):
    # Random city coordinates
    random.seed(42)
    np.random.seed(42)

    # Generate a random graph with 8 vertices
    graph = generate_graph_with_min_degree_2(num_cities, prob)
    draw_graph(graph)

    # Initialize solver
    INPUT_DIM = num_cities
    # HIDDEM_DIM = 64 # no more hidden dim lol
    solver = TSPSolver(num_cities, input_dim=INPUT_DIM)
    optimizer = optim.Adam(solver.parameters(), lr = lr)

    weight_matrix = get_weight_matrix(graph)

    # Solve TSP
    best_tour, best_length = cross_entropy_method(solver, weight_matrix, optimizer, num_iterations=iters, population_size=pop, elite_fraction=fraction)

    print("---------------------")
    print(f"Best Tour: {best_tour}")
    print(f"Tour Length: {best_length}\n")

    if tsp_solver:
      print("\n---------------------")
      print("TSP Solver solution:")
      best_path, min_cost = solve_tsp_dynamic_programming(weight_matrix)
      print("Best path found:", best_path)
      print("Minimum cost:", min_cost)

    if tsp_heuristics:
      print("\n---------------------")
      print("TSP Heuristics solution:")
      best_path, min_cost = solve_tsp_simulated_annealing(weight_matrix)
      print("Best path found:", best_path)
      print("Minimum cost:", min_cost)

# Testing the result
CE_TSP(num_cities=8, prob = 0.8, iters=10, pop = 50, lr=0.01, tsp_solver=True, tsp_heuristics=True)



## Problem 1c.(2 points)
Now let's test what we got. Run ``CE_TSP`` with 15 cities, sample population of 1000 twice: with elite fraction 10% and elite fraction 0.5%. Compare the results. You can pick ``prob`` as you like, but low ``prob`` will result in more boring graphs.

Before you run, try to predict which elite fraction will make the iterations to decrease tour length faster. After you see the results, write what was your prediction and how it relates to the results you saw. Try to explain the results.

In [ ]:
CE_TSP(???)
CE_TSP(???)

##Problem 1d.(2points)
Now let's inspect the effect of learning rate. Run ``CE_TSP`` with the same number of cities, same population, elite fraction 1% twice: with ``lr=0.01`` and with ``lr=3e-4``. Explain the results.

In [ ]:
CE_TSP(???)
CE_TSP(???)

##Problem 1e.(2 points)
### Solving a Larger TSP Instance

Now that we have a better understanding of the parameters, let’s tackle a more challenging instance.

- **Number of cities**: 50  
- **Edge probability**: 0.8  
- **Other parameters**: Choose reasonable values for:
  - Population size  
  - Number of iterations  
  - Elite fraction  
  - Learning rate  

Run the function `CE_TSP` with these settings.

#### Goal
Try to outperform the built-in TSP heuristics by setting `tsp_heuristics=True`.  
Don’t stress if you don’t beat it—achieving a solution within **$1.2 \times$** the heuristic score is considered a success.

> **Note:**  
> It’s strongly recommended to set `tsp_solver=False`, as the exact solver is likely to fail or freeze on problems of this size.

#### Caution
This experiment can be computationally expensive, with runtime ranging from **tens of minutes to over an hour**. Choose your parameters wisely—or be prepared to wait.

In [ ]:
CE_TSP(num_cities=50, prob=0.8, ???, tsp_solver=False, tsp_heuristics=True)